In [ ]:
import numpy as np
from PIL import Image, ImageDraw
import cv2  # OpenCV for webcam capture
import matplotlib.pyplot as plt

# Initialize webcam capture
cap = cv2.VideoCapture(0)  # 0 is the default webcam

print("Press 's' to capture an image.")

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break
    
    # Display the resulting frame
    cv2.imshow('Webcam', frame)
    
    # Wait for the 's' key to be pressed
    if cv2.waitKey(1) & 0xFF == ord('s'):
        print("Image captured!")
        break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()

# Convert the frame from BGR (OpenCV format) to RGB (PIL format)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Convert the frame to a PIL Image
img = Image.fromarray(frame)

# Convert the image to grayscale
gray_img = img.convert('L')

# Convert the grayscale image to a numpy array
img_array = np.array(gray_img)

# Get the dimensions of the image
height, width = img_array.shape

# Number of columns and rows
num_cols = 200
num_rows = 50

# Calculate the size of each grid cell
cell_width = width // num_cols
cell_height = height // num_rows

# Initialize an array to store intensity values
intensities = np.zeros((num_rows, num_cols))

# Calculate the intensity for each grid cell
for row in range(num_rows):
    for col in range(num_cols):
        start_row = row * cell_height
        end_row = min((row + 1) * cell_height, height)
        start_col = col * cell_width
        end_col = min((col + 1) * cell_width, width)
        
        # Extract the grid cell
        grid_cell = img_array[start_row:end_row, start_col:end_col]
        
        # Calculate the mean intensity of the grid cell
        intensities[row, col] = np.mean(grid_cell)

# Calculate column-wise projection
column_projection = np.sum(intensities, axis=0)

# Calculate the difference between consecutive columns
column_diff = np.diff(column_projection)

# Set a threshold for detecting sudden increases
increase_threshold = 20  # Adjust this threshold based on your needs

# Detect columns with a sudden increase in intensity
defect_columns = [i+1 for i, diff in enumerate(column_diff) if diff > increase_threshold]

# Group consecutive defective columns
defect_groups = []
current_group = []

for col in defect_columns:
    if current_group and col == current_group[-1] + 1:
        current_group.append(col)
    else:
        if current_group:
            defect_groups.append(current_group)
        current_group = [col]

if current_group:
    defect_groups.append(current_group)

# Draw bounding boxes on defective regions
img_with_boxes = gray_img.convert("RGB")  # Convert to RGB to draw colored boxes
draw = ImageDraw.Draw(img_with_boxes)

for group in defect_groups:
    start_col = group[0] * cell_width
    end_col = (group[-1] + 1) * cell_width
    draw.rectangle([start_col, 0, end_col, height], outline="red", width=2)

# Display the grayscale image with bounding boxes and the plot
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.imshow(img_with_boxes)
plt.title('Grayscale Image with Defect Bounding Boxes')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.plot(range(num_cols), column_projection, marker='o', linestyle='-')
plt.title('Sum of Intensities vs. Column Number')
plt.xlabel('Column Number')
plt.ylabel('Sum of Intensities')
plt.grid(True)

# Mark the defect regions on the plot
for group in defect_groups:
    plt.axvspan(group[0], group[-1], color='red', alpha=0.3)

plt.tight_layout()
plt.show()
